In [1]:
import os, sys
import pandas as pd
import numpy as np
import openpyxl
from sklearn.preprocessing import MinMaxScaler
from slugify import slugify

In [2]:
# path configurator

path_m = r'C:\\Users\\user\\Desktop\\10\\'
events_by_cat = path_m+'5_Типы событий, регистрируемых по типу объекта многоквартирный дом.xlsx'
work_types_by_support = path_m+'5_Типы событий, регистрируемых по типу объекта многоквартирный дом.xlsx'
work_types_by_cap = path_m+'4_Виды работ по капитальному ремонту многоквартирных домов.xlsx'
works_done = path_m+'3_Работы по капитальному ремонту, проведенные в многоквартирных домах.xlsx'
incidents = path_m+'2_Инциденты,_зарегистрированные_на_объектах_городского_хозяйства.xlsx'
multi_houses_characts = path_m+'1_Многоквартирные дома с технико-экономическими характеристиками.xlsx'

In [87]:
# function to preprocess some russian strings to slugs

def slug_me(str_inp):
    if type(str_inp) == str:
        str_inp = str_inp.replace('Российская Федерация, город Москва, ', '')
        if ', к.' in str_inp:
            str_inp = str_inp.split(',')[0:2]
            str_inp = ''.join(str_inp)
        if 'муниципальный округ' in str_inp:
            str_inp = str_inp.split(',')[1:3]
            str_inp = ''.join(str_inp)
        str_inp = str_inp.replace('Дом по адресу ', '')
        str_inp = str_inp.replace('Адрес ', '')
        str_inp = str_inp.replace('дом ', '')
        str_inp = str_inp.replace('район ', '')
        str_inp = str_inp.replace('улица ', '')
        str_inp = str_inp.replace('ул.', '')
        str_inp = str_inp.replace('д.', '')
        str_inp = str_inp.replace(',', '')
        rez = slugify(str_inp, allow_unicode=False, lowercase=True, save_order=True, separator="_")
        return rez
    else:
        rez = 'err'

print(slug_me("Дом по адресу Красковская ул., д.121А"))
print(slug_me("Дом по адресу Рокоссовского Маршала бульв., д.7, к.4"))
print(slug_me("внутригородская территория муниципальный округ Косино-Ухтомский, улица Дмитриевского, дом 5"))
print(slug_me("внутригородская территория муниципальный округ Преображенское, Большая Черкизовская улица, дом 30, корпус 5"))
print(slug_me("Российская Федерация, город Москва, внутригородская территория муниципальный округ Богородское, Краснобогатырская улица, дом 21"))

kraskovskaia_121a
rokossovskogo_marshala_bulv_7
dmitrievskogo_5
bolshaia_cherkizovskaia_30
krasnobogatyrskaia_21


In [105]:
# load xlsx datasets to mem

df_incidents = pd.read_excel(incidents)
df_multi_houses_characts = pd.read_excel(multi_houses_characts)
df_work_types_by_cap = pd.read_excel(work_types_by_cap)
df_events_by_cat = pd.read_excel(events_by_cat)
df_works_done = pd.read_excel(works_done)

UsageError: Line magic function `%%timeit` not found.


In [130]:
# create a tmp dataframes to avoid changing of originals

df_incidents_tmp = df_incidents
df_multi_houses_characts_tmp = df_multi_houses_characts
df_work_types_by_cap_tmp = df_work_types_by_cap
df_events_by_cat_tmp = df_events_by_cat
df_works_done_tmp = df_works_done

# filling NaNs with nulls

df_incidents_tmp = df_incidents_tmp.fillna(0)
df_multi_houses_characts_tmp = df_multi_houses_characts_tmp.fillna(0)
df_work_types_by_cap_tmp = df_work_types_by_cap_tmp.fillna(0)
df_events_by_cat_tmp = df_events_by_cat_tmp.fillna(0)
df_works_done_tmp = df_works_done_tmp.fillna(0)

In [ ]:
print(df_tmp.describe())

## 1_Многоквартирные дома с технико-экономическими характеристиками.xlsx

In [138]:
df_multi_houses_characts_tmp = df_multi_houses_characts_tmp.iloc[1:] # to remove russian captions from ds

names = {
            'COL_754': 'dest', # Назначение
            'COL_756': 'build_year', # Год постройки
            'COL_757': 'reconst_year', # Год реконструкции
            'COL_755': 'ownership', # Форма собственности
            'COL_759': 'floars', # Количество этажей
            'COL_760': 'entrances', # Количество подъездов
            'COL_761': 'apartments', # Количество квартир
            'COL_782': 'unom', # unom
            'COL_758': 'project_series', # Серия проекта
            'COL_762': 'total_area', # Общая площадь
            'COL_763': 'living_area', # Общая площадь жилых помещений
            'COL_764': 'nonliving_area', # Общая площадь нежилых помещений
            'COL_765': 'construction_volume', # Строительный объем
            'COL_766': 'depreciation_object', # Износ объекта (по БТИ)
            'COL_767': 'energyefficiency_class', # Класс энергоэффективности
            'COL_769': 'wall_material', # Материал стен
            'COL_770': 'sign_building_failure', # Признак аварийности здания
            'COL_771': 'passenger_elevators',  # Количество пассажирских лифтов
            'COL_3363': 'freight_elevators', # Количество грузовых лифтов
            'COL_772': 'passenger_freight_elevators', # Количество грузопассажирских лифтов
            'COL_775': 'roof_cleaning', # Очередность уборки кровли
            'COL_2156': 'type_social_object', # Вид социального объекта
            'COL_2463': 'type_housing_stock', # Тип жилищного фонда
            'COL_3163': 'status_MKD', # Статус МКД
            'COL_3243': 'management_status_MKD', # Статус управления МКД
            'COL_3468': 'reason_changing_status_MKD', # Причина Изменения Статуса МКД
            'COL_103506': 'category_MKD', # Категория МКД
         }

# rename column names
df_multi_houses_characts_tmp = df_multi_houses_characts_tmp.rename(columns=names)
df_multi_houses_characts_tmp['NAME'] = df_multi_houses_characts_tmp['NAME'].apply(slug_me)
df_multi_houses_characts_tmp['dest'] = df_multi_houses_characts_tmp['dest'].apply(slug_me)
df_multi_houses_characts_tmp['unom'] = df_multi_houses_characts_tmp['unom'].apply(int)

In [139]:
df_multi_houses_characts_tmp.head()

,ID,NAME,PARENT_ID,LOGIN,dest,ownership,build_year,reconst_year,project_series,floars,...,roof_cleaning,COL_781,unom,type_social_object,type_housing_stock,status_MKD,management_status_MKD,freight_elevators,reason_changing_status_MKD,category_MKD
1,1500923.0,kraskovskaia_121a,0.0,WS,None,0,1956,0,2048798,1,...,0,22289204,49928,0,22741963,58761330,0,0,0,0
2,1500933.0,baikalskaia_18,0.0,WS,None,0,2005,0,2048798,18,...,0,22289201,64148,0,42875644,58761330,45063584,0,0,0
3,1500943.0,perovskaia_48,0.0,WS,None,0,1955,0,2048912,4,...,22289162,22289204,19485,0,42875644,58761330,45063584,0,0,0
4,1500951.0,parkovaia_3_ia_39,0.0,404070001,None,0,1960,0,2048912,5,...,22289162,22289201,18533,0,42875644,58761330,45063584,0,0,0
5,1500954.0,svobodnyi_prosp_11,0.0,WS,None,0,1973,0,2048755,9,...,0,22289201,22830,0,42875644,58761330,45063584,0,0,0


In [103]:
for c in df_multi_houses_characts_tmp.columns:
    print(c, end=' ')

ID NAME PARENT_ID LOGIN dest ownership build_year reconst_year project_series floars entrances apartments total_area living_area nonliving_area construction_volume depreciation_object energyefficiency_class wall_material sign_building_failure passenger_elevators passenger_freight_elevators roof_cleaning COL_781 unom type_social_object type_housing_stock status_MKD management_status_MKD freight_elevators reason_changing_status_MKD category_MKD 

## 2_Инциденты,_зарегистрированные_на_объектах_городского_хозяйства.xlsx

In [140]:
names = {
            'Наименование': 'incident_name', # Наименование
            'Источник': 'source', # Источник
            'Дата создания во внешней системе': 'external_create_date', # Дата создания во внешней системе
            'Дата закрытия': 'close_date', # Дата закрытия
            'Адрес': 'address', # Адрес
            'Округ': 'district', # Округ
            'Дата и время завершения события во': 'done_date', # Дата и время завершения события во
         }

# rename column names
df_incidents_tmp = df_incidents_tmp.rename(columns=names)

# sluglify names of incidents, addresses and districts
df_incidents_tmp['incident_name'] = df_incidents_tmp['incident_name'].apply(slug_me)
df_incidents_tmp['address'] = df_incidents_tmp['address'].apply(slug_me)
df_incidents_tmp['district'] = df_incidents_tmp['district'].apply(slug_me)

# prepare date format
df_incidents_tmp['external_create_date'] = pd.to_datetime(df_incidents_tmp['external_create_date'], errors='coerce')
df_incidents_tmp['external_create_date_year'] = df_incidents_tmp['external_create_date'].dt.year
df_incidents_tmp['external_create_date_month'] = df_incidents_tmp['external_create_date'].dt.month
df_incidents_tmp['external_create_date_day'] = df_incidents_tmp['external_create_date'].dt.day
df_incidents_tmp['done_date'] = pd.to_datetime(df_incidents_tmp['done_date'], errors='coerce')
df_incidents_tmp['done_date_year'] = df_incidents_tmp['done_date'].dt.year
df_incidents_tmp['done_date_month'] = df_incidents_tmp['done_date'].dt.month
df_incidents_tmp['done_date_day'] = df_incidents_tmp['done_date'].dt.day
df_incidents_tmp['close_date'] = pd.to_datetime(df_incidents_tmp['close_date'], errors='coerce')
df_incidents_tmp['close_date_year'] = df_incidents_tmp['close_date'].dt.year
df_incidents_tmp['close_date_month'] = df_incidents_tmp['close_date'].dt.month
df_incidents_tmp['close_date_day'] = df_incidents_tmp['close_date'].dt.day

# remove unnecsessary
df_incidents_tmp.drop('external_create_date', axis=1, inplace=True)
df_incidents_tmp.drop('close_date', axis=1, inplace=True)
df_incidents_tmp.drop('done_date', axis=1, inplace=True)
df_incidents_tmp.drop('district', axis=1, inplace=True)

KeyError: "['LOGIN'] not found in axis"

In [141]:
df_incidents_tmp.head()

,incident_name,source,address,unom,external_create_date_year,external_create_date_month,external_create_date_day,done_date_year,done_date_month,done_date_day,close_date_year,close_date_month,close_date_day
0,otkryt_kolodets,MVK,malaia_semionovskaia_13,2115191,2022,8,30,1970,1,1,2022.0,8.0,30.0
1,otklonenie_gvs_nizhe_normy_nochiu_monitoring,ASUPR,dmitrievskogo_5,2404461,2022,2,20,1970,1,1,2022.0,4.0,4.0
2,otklonenie_gvs_nizhe_normy_dnem_monitoring,ASUPR,dmitrievskogo_5,2404461,2022,3,7,1970,1,1,2022.0,3.0,9.0
3,temperatura_gvs_nizhe_normy,ASUPR,dmitrievskogo_5,2404461,2022,3,5,1970,1,1,2022.0,3.0,5.0
4,temperatura_gvs_nizhe_normy,ASUPR,dmitrievskogo_5,2404461,2022,2,18,1970,1,1,2022.0,2.0,18.0


## 3_Работы по капитальному ремонту, проведенные в многоквартирных домах.xlsx

In [132]:
# prepare date format
df_works_done_tmp['PLAN_DATE_START'] = pd.to_datetime(df_works_done_tmp['PLAN_DATE_START'], dayfirst=True)
df_works_done_tmp['PLAN_DATE_START_year'] = df_works_done_tmp['PLAN_DATE_START'].dt.year
df_works_done_tmp['PLAN_DATE_START_month'] = df_works_done_tmp['PLAN_DATE_START'].dt.month
df_works_done_tmp['PLAN_DATE_START_day'] = df_works_done_tmp['PLAN_DATE_START'].dt.day

df_works_done_tmp['PLAN_DATE_END'] = pd.to_datetime(df_works_done_tmp['PLAN_DATE_END'], dayfirst=True)
df_works_done_tmp['PLAN_DATE_END_year'] = df_works_done_tmp['PLAN_DATE_END'].dt.year
df_works_done_tmp['PLAN_DATE_END_month'] = df_works_done_tmp['PLAN_DATE_END'].dt.month
df_works_done_tmp['PLAN_DATE_END_day'] = df_works_done_tmp['PLAN_DATE_END'].dt.day

df_works_done_tmp['FACT_DATE_START'] = pd.to_datetime(df_works_done_tmp['FACT_DATE_START'], dayfirst=True)
df_works_done_tmp['FACT_DATE_START_year'] = df_works_done_tmp['FACT_DATE_START'].dt.year
df_works_done_tmp['FACT_DATE_START_month'] = df_works_done_tmp['FACT_DATE_START'].dt.month
df_works_done_tmp['FACT_DATE_START_day'] = df_works_done_tmp['FACT_DATE_START'].dt.day

df_works_done_tmp['FACT_DATE_END'] = pd.to_datetime(df_works_done_tmp['FACT_DATE_END'], dayfirst=True)
df_works_done_tmp['FACT_DATE_END_year'] = df_works_done_tmp['FACT_DATE_END'].dt.year
df_works_done_tmp['FACT_DATE_END_month'] = df_works_done_tmp['FACT_DATE_END'].dt.month
df_works_done_tmp['FACT_DATE_END_day'] = df_works_done_tmp['FACT_DATE_END'].dt.day

df_works_done_tmp['unom'] = df_works_done_tmp['UNOM']

# remove unnecsessary
df_works_done_tmp.drop('PLAN_DATE_START', axis=1, inplace=True)
df_works_done_tmp.drop('PLAN_DATE_END', axis=1, inplace=True)
df_works_done_tmp.drop('FACT_DATE_END', axis=1, inplace=True)
df_works_done_tmp.drop('FACT_DATE_START', axis=1, inplace=True)

# sluglify names
df_works_done_tmp['WORK_NAME'] = df_works_done_tmp['WORK_NAME'].apply(slug_me)
df_works_done_tmp['AdmArea'] = df_works_done_tmp['AdmArea'].apply(slug_me)
df_works_done_tmp['District'] = df_works_done_tmp['District'].apply(slug_me)
df_works_done_tmp['Address'] = df_works_done_tmp['Address'].apply(slug_me)

In [133]:
df_works_done_tmp

,global_id,PERIOD,WORK_NAME,NUM_ENTRANCE,ElevatorNumber,AdmArea,District,Address,UNOM,PLAN_DATE_START_year,...,PLAN_DATE_END_year,PLAN_DATE_END_month,PLAN_DATE_END_day,FACT_DATE_START_year,FACT_DATE_START_month,FACT_DATE_START_day,FACT_DATE_END_year,FACT_DATE_END_month,FACT_DATE_END_day,unom
0,351060,2022,zamena_liftovogo_oborudovaniia,1.0,1,vostochnyi_administrativnyi_okrug,bogorodskoe,ivanteevskaia_1,20151,2022,...,2022,8,28,2022,6,27,2022,10,14,20151
1,430496,2022,zamena_liftovogo_oborudovaniia,1.0,1,vostochnyi_administrativnyi_okrug,bogorodskoe,ivanteevskaia_1,20151,2022,...,2022,10,14,2022,9,30,2022,10,21,20151
2,348580,2022,remont_vnutridomovykh_inzhenernykh_sistem_vodo...,0.0,0,vostochnyi_administrativnyi_okrug,bogorodskoe,krasnobogatyrskaia_21,11466,2022,...,2022,6,20,2022,2,1,2022,5,25,11466
3,348581,2022,remont_vnutridomovykh_inzhenernykh_sistem_tepl...,0.0,0,vostochnyi_administrativnyi_okrug,bogorodskoe,krasnobogatyrskaia_21,11466,2022,...,2022,7,10,2022,5,7,2022,8,3,11466
4,348578,2022,remont_vnutridomovykh_inzhenernykh_sistem_khol...,0.0,0,vostochnyi_administrativnyi_okrug,bogorodskoe,krasnobogatyrskaia_21,11466,2022,...,2022,6,20,2022,2,1,2022,6,27,11466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1742,366525,2022,remont_vnutridomovykh_inzhenernykh_sistem_tepl...,0.0,0,vostochnyi_administrativnyi_okrug,golianovo,kamchatskaia_4,9177,2022,...,2022,7,10,2022,3,7,2022,8,11,9177
1743,366526,2022,remont_podvalnykh_pomeshchenii_otnosiashchikhs...,0.0,0,vostochnyi_administrativnyi_okrug,golianovo,kamchatskaia_4,9177,2022,...,2022,9,30,2022,2,4,2022,6,17,9177
1744,366523,2022,remont_vnutridomovykh_inzhenernykh_sistem_vodo...,0.0,0,vostochnyi_administrativnyi_okrug,golianovo,kamchatskaia_4,9177,2022,...,2022,5,31,2022,2,4,2022,8,4,9177
1745,394724,2022,remont_vnutrennego_vodostoka,0.0,0,vostochnyi_administrativnyi_okrug,ivanovskoe,bolshoi_kupavenskii_proezd_10,12178,2022,...,2022,12,31,2022,10,10,2022,11,30,12178


## 4_Виды работ по капитальному ремонту многоквартирных домов.xlsx

In [35]:
df_work_types_by_cap_tmp = df_work_types_by_cap_tmp.iloc[1:] # to remove russian captions from ds

names = {
            df_work_types_by_cap_tmp.columns[0]: 'index', # №\nп/п
            df_work_types_by_cap_tmp.columns[1]: 'code', # Код
            df_work_types_by_cap_tmp.columns[2]: 'work_name', # Наименование 	
            df_work_types_by_cap_tmp.columns[3]: 'owner_name', # Наименование объекта общего имущества
            df_work_types_by_cap_tmp.columns[4]: 'work_type', # Тип работ
            df_work_types_by_cap_tmp.columns[5]: 'work_group', # Группа работ
            df_work_types_by_cap_tmp.columns[6]: 'slug_work', # Сокращенное наименование работы
         }

# rename column names
df_work_types_by_cap_tmp = df_work_types_by_cap_tmp.rename(columns=names)
df_work_types_by_cap_tmp['work_name'] = df_work_types_by_cap_tmp['work_name'].apply(slug_me)
df_work_types_by_cap_tmp['owner_name'] = df_work_types_by_cap_tmp['owner_name'].apply(slug_me)
df_work_types_by_cap_tmp['work_type'] = df_work_types_by_cap_tmp['work_type'].apply(slug_me)
df_work_types_by_cap_tmp['work_group'] = df_work_types_by_cap_tmp['work_group'].apply(slug_me)
df_work_types_by_cap_tmp['slug_work'] = df_work_types_by_cap_tmp['slug_work'].apply(slug_me)

In [ ]:
df_work_types_by_cap_tmp

### Объединяем датасет выполненных работ с инцидентами

In [142]:
df_merged = df_works_done_tmp.merge(df_incidents_tmp, on='unom', how='left')

### Объединяем датасет выполненных и инцидентов с характеристиками об объектах

In [150]:
df_merged = df_merged.merge(df_multi_houses_characts_tmp, on='unom', how='left')

### дампим комбинированный новый (комплексный) датасет

In [153]:
df_merged.to_csv('complex_df.csv')

### Конвертим категоризированные данные Скейлим данные

In [154]:
new_df_tmp = pd.get_dummies(df_merged)

In [162]:
for c in new_df_tmp.columns:
    print(c, end=',  ')

global_id,  PERIOD,  NUM_ENTRANCE,  UNOM,  PLAN_DATE_START_year,  PLAN_DATE_START_month,  PLAN_DATE_START_day,  PLAN_DATE_END_year,  PLAN_DATE_END_month,  PLAN_DATE_END_day,  FACT_DATE_START_year,  FACT_DATE_START_month,  FACT_DATE_START_day,  FACT_DATE_END_year,  FACT_DATE_END_month,  FACT_DATE_END_day,  unom,  external_create_date_year,  external_create_date_month,  external_create_date_day,  done_date_year,  done_date_month,  done_date_day,  close_date_year,  close_date_month,  close_date_day,  ID,  PARENT_ID,  WORK_NAME_remont_fasadov,  WORK_NAME_remont_kryshi,  WORK_NAME_remont_musoroprovoda,  WORK_NAME_remont_podezdov_napravlennyi_na_vosstanovlenie_ikh_nadlezhashchego_sostoianiia_i_provodimyi_pri_vypolnenii_inykh_rabot_po_kapitalnomu_remontu_obshchego_imushchestva_v_mnogokvartirnom_dome,  WORK_NAME_remont_podvalnykh_pomeshchenii_otnosiashchikhsia_k_obshchemu_imushchestvu_v_mnogokvartirnom_dome,  WORK_NAME_remont_pozharnogo_vodoprovoda,  WORK_NAME_remont_vnutrennego_vodostoka,  WO

In [ ]:
scaler = MinMaxScaler()
new_df_tmp = pd.DataFrame(scaler.fit_transform(new_df_tmp), columns=new_df_tmp.columns)
new_df_tmp = new_df_tmp.fillna(0)

In [ ]:
new_df_tmp = pd.get_dummies(df_tmp)
scaler = MinMaxScaler()
new_df_tmp = pd.DataFrame(scaler.fit_transform(new_df_tmp), columns=new_df_tmp.columns)
new_df_tmp = new_df_tmp.fillna(0)

new_obj_df_tmp = pd.get_dummies(df_obj_tmp)
scaler2 = MinMaxScaler()
new_obj_df_tmp = pd.DataFrame(scaler2.fit_transform(new_obj_df_tmp), columns=new_obj_df_tmp.columns)
new_obj_df_tmp = new_obj_df_tmp.fillna(0)

In [ ]:
df_obj_tmp.head

In [ ]:
print(new_obj_df_tmp.dtypes)

In [ ]:
for c in new_obj_df_tmp.columns:
    print(c)

In [ ]:
merged_df = new_obj_df_tmp.merge(new_df_tmp, on='unom', how='inner')

In [ ]:
for col in df_tmp:
    print(df_tmp[col].unique())

In [ ]:
new_df = df_tmp[df_tmp['source'] == 'EDC'].copy()
new_df

In [ ]:
for col in df_tmp:
    print(new_df[col].unique())

In [ ]:
print(new_df_tmp.columns)
new_df_tmp.to_csv('tmp.csv')

In [ ]:
from sklearn.cluster import KMeans

# Определение числа кластеров (можно изменить в зависимости от ваших потребностей)
n_clusters = 5

# Создание модели KMeans
kmeans = KMeans(n_clusters=n_clusters)

# Обучение модели и присвоение меток кластеров объектам
new_df_tmp['cluster'] = kmeans.fit_predict(new_df_tmp)

# Вывод первых нескольких строк данных с метками кластеров
print(new_df_tmp.head())

In [ ]:
df_obj_tmp.describe()
for col in df_obj_tmp:
    print(df_obj_tmp[col].unique())

In [ ]:
df_obj_tmp['ID'] = df_obj_tmp['ID'].astype(int)
df_obj_tmp = pd.get_dummies(df_obj_tmp)
scaler = MinMaxScaler()
df_obj_tmp = pd.DataFrame(scaler.fit_transform(df_obj_tmp), columns=df_obj_tmp.columns)
df_obj_tmp

In [ ]:
df_merged = pd.concat([df_tmp, df_obj_tmp], ignore_index=True, sort=False)
df_merged

In [ ]:
df_houses_by_unom = df_multi_houses_characts_tmp.loc[df_multi_houses_characts_tmp['unom'] == '20151']
df_houses_by_unom.head()

In [ ]:
incidents_by_unom = df_incidents_tmp.loc[df_incidents_tmp['unom'] == 2404461]
incidents_by_unom.head()

In [ ]:
complex_df = df_works_done_tmp

for unom in df_works_done_tmp['UNOM']:
    incidents_by_unom = df_incidents_tmp.loc[df_incidents_tmp['unom'] == unom]

    if len(incidents_by_unom) > 0:
        complex_df['incident_name'] = incidents_by_unom['incident_name']
        complex_df['source'] = incidents_by_unom['source']
        complex_df['external_create_date_year'] = incidents_by_unom['external_create_date_year']
        complex_df['external_create_date_month'] = incidents_by_unom['external_create_date_month']
        complex_df['external_create_date_day'] = incidents_by_unom['external_create_date_day']
        complex_df['done_date_year'] = incidents_by_unom['done_date_year']
        complex_df['done_date_month'] = incidents_by_unom['done_date_month']
        complex_df['done_date_day'] = incidents_by_unom['done_date_day']
        complex_df['close_date_year'] = incidents_by_unom['close_date_year']
        complex_df['close_date_month'] = incidents_by_unom['close_date_month']
        complex_df['close_date_day'] = incidents_by_unom['close_date_day']
        
    # type_housing_stock status_MKD management_status_MKD freight_elevators reason_changing_status_MKD category_MKD 
    objects_by_unom = df_multi_houses_characts_tmp.loc[df_multi_houses_characts_tmp['unom'] == str(unom)]
    if len(objects_by_unom) > 0:
        complex_df['build_year'] = objects_by_unom['build_year']
        complex_df['ownership'] = objects_by_unom['ownership']
        complex_df['reconst_year'] = objects_by_unom['reconst_year']
        complex_df['project_series'] = objects_by_unom['project_series']
        complex_df['floars'] = objects_by_unom['floars']
        complex_df['entrances'] = objects_by_unom['entrances']
        complex_df['apartments'] = objects_by_unom['apartments']
        complex_df['total_area'] = objects_by_unom['total_area']
        complex_df['living_area'] = objects_by_unom['living_area']
        complex_df['nonliving_area'] = objects_by_unom['nonliving_area']
        complex_df['construction_volume'] = objects_by_unom['construction_volume']
        complex_df['nonliving_area'] = objects_by_unom['nonliving_area']
        complex_df['depreciation_object'] = objects_by_unom['depreciation_object']
        complex_df['energyefficiency_class'] = objects_by_unom['energyefficiency_class']
        complex_df['wall_material'] = objects_by_unom['wall_material']
        complex_df['sign_building_failure'] = objects_by_unom['sign_building_failure']
        complex_df['passenger_elevators'] = objects_by_unom['passenger_elevators']
        complex_df['passenger_freight_elevators'] = objects_by_unom['passenger_freight_elevators']
        complex_df['roof_cleaning'] = objects_by_unom['roof_cleaning']
        complex_df['passenger_freight_elevators'] = objects_by_unom['passenger_freight_elevators']
        complex_df['COL_781'] = objects_by_unom['COL_781']
        complex_df['type_social_object'] = objects_by_unom['type_social_object']
        complex_df['type_housing_stock'] = objects_by_unom['type_housing_stock']
        complex_df['status_MKD'] = objects_by_unom['status_MKD']
        complex_df['management_status_MKD'] = objects_by_unom['management_status_MKD']
        complex_df['freight_elevators'] = objects_by_unom['freight_elevators']
        complex_df['reason_changing_status_MKD'] = objects_by_unom['reason_changing_status_MKD']
        complex_df['category_MKD'] = objects_by_unom['category_MKD']
        
print(complex_df.head())
complex_df.to_csv('complex_df.csv')